#### LightGBMでモデル構築

In [1]:
import os
os.chdir('/Users/kinoshitashouhei/Desktop/competitions/05_Prob_Space/Kiva/')

In [23]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import gc
import lightgbm as lgb
import pickle
import optuna
import warnings

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, train_test_split
from sklearn.pipeline import Pipeline

# 自作関数のインポート
from functions.common import *
from functions import preprocessing

warnings.filterwarnings('ignore')

In [3]:
# 各種設定
N_COMP = 10
DICT_DTYPE = {'LOAN_ID': 'str', 'IMAGE_ID': 'str'}
LIST_COL_DROPS = [COL_LOAN_ID,
                  COL_DESCRIPTION,
                  COL_DESCRIPTION_TRANSLATED,
                  COL_ACTIVITY_NAME,
                  COL_LOAN_USE,
                  COL_IMAGE_ID,
                  COL_TOWN_NAME,
                  COL_COUNTRY_NAME,
                  COL_TAGS]

LIST_TARGET_ENC = [COL_ORIGINAL_LANGUAGE]
LIST_LABEL_ENC = [COL_SECTOR_NAME, COL_CURRENCY_POLICY, COL_CURRENCY, COL_REPAYMENT_INTERVAL, COL_DISTRIBUTION_MODEL]
LIST_LDA_DESCRIPTINO_TRANSLATED = [f'COL_LDA_DESCRIPTION_TRANSLATED_{i+1}' for i in range(N_COMP)]
LIST_LDA_LOAN_USE = [f'COL_LDA_LOAN_USE_{i+1}' for i in range(N_COMP)]

In [4]:
# モデルのパス
LDA_PATH_DESCRIPTION_TRANSLATED = 'models/lda_description_translated.pkl'
LDA_PATH_LOAN_USE = 'models/lda_loan_use.pkl'

# モデルの読み込み
with open(LDA_PATH_DESCRIPTION_TRANSLATED, mode='rb') as fp:
    CLF_LDA_DESCRIPTION_TRANSLATED = pickle.load(fp)
    
with open(LDA_PATH_LOAN_USE, mode='rb') as fp:
    CLF_LDA_LOAN_USE = pickle.load(fp)

In [5]:
# csvデータの読み込み
df_train = pd.read_csv('data/train.csv', dtype=DICT_DTYPE)
df_test = pd.read_csv('data/test.csv', dtype=DICT_DTYPE)

In [6]:
# 欠損値の補完
df_train = preprocessing.fill_na_DESCRIPTION_TRANSLATED(df_train)

# テキストを前処理(DESCRIPTION_TRANSLATED)
df_train[COL_DESCRIPTION_TRANSLATED] = df_train[COL_DESCRIPTION_TRANSLATED].apply(preprocessing.replace_str)
df_train[COL_DESCRIPTION_TRANSLATED] = df_train[COL_DESCRIPTION_TRANSLATED].apply(preprocessing.lower_text)
df_test[COL_DESCRIPTION_TRANSLATED] = df_test[COL_DESCRIPTION_TRANSLATED].apply(preprocessing.replace_str)
df_test[COL_DESCRIPTION_TRANSLATED] = df_test[COL_DESCRIPTION_TRANSLATED].apply(preprocessing.lower_text)

# テキストを前処理(LOAN_USE)
df_train[COL_LOAN_USE] = df_train[COL_LOAN_USE].apply(preprocessing.replace_str_to_use)
df_train[COL_LOAN_USE] = df_train[COL_LOAN_USE].apply(preprocessing.lower_text)
df_test[COL_LOAN_USE] = df_test[COL_LOAN_USE].apply(preprocessing.replace_str_to_use)
df_test[COL_LOAN_USE] = df_test[COL_LOAN_USE].apply(preprocessing.lower_text)

# ターゲットエンコーディング
df_train, df_test = preprocessing.target_encoding_oof(df_train, df_test, LIST_TARGET_ENC, COL_LOAN_AMOUNT)

In [7]:
# 単語のカウント行列(DESCRIPTION_TRANSLATED)
text_vec_description_translated = CountVectorizer()
text_vec_description_translated.fit(df_train[COL_DESCRIPTION_TRANSLATED])
bow_description_translated_train = text_vec_description_translated.transform(df_train[COL_DESCRIPTION_TRANSLATED])
bow_description_translated_test = text_vec_description_translated.transform(df_test[COL_DESCRIPTION_TRANSLATED])

# 単語のカウント行列(LOAN_USE)
text_vec_loan_use = CountVectorizer()
text_vec_loan_use.fit(df_train[COL_LOAN_USE])
bow_loan_use_train = text_vec_loan_use.transform(df_train[COL_LOAN_USE])
bow_loan_use_test = text_vec_loan_use.transform(df_test[COL_LOAN_USE])

In [8]:
# 学習済みのモデルを適用(DESCRIPTION_TRANSLATED)
transformed_lda_description_translated_train = CLF_LDA_DESCRIPTION_TRANSLATED.transform(bow_description_translated_train)
df_transformed_lda = pd.DataFrame(data=transformed_lda_description_translated_train, columns=LIST_LDA_DESCRIPTINO_TRANSLATED)
df_train = pd.concat([df_train, df_transformed_lda], axis=1)

del transformed_lda_description_translated_train, df_transformed_lda
gc.collect()

transformed_lda_description_translated_test = CLF_LDA_DESCRIPTION_TRANSLATED.transform(bow_description_translated_test)
df_transformed_lda = pd.DataFrame(data=transformed_lda_description_translated_test, columns=LIST_LDA_DESCRIPTINO_TRANSLATED)
df_test = pd.concat([df_test, df_transformed_lda], axis=1)

del transformed_lda_description_translated_test, df_transformed_lda
gc.collect()

24

In [9]:
# 学習済みのモデルを適用(LOAN_USE)
transformed_lda_loan_use_train = CLF_LDA_LOAN_USE.transform(bow_loan_use_train)
df_transformed_lda = pd.DataFrame(data=transformed_lda_loan_use_train, columns=LIST_LDA_LOAN_USE)
df_train = pd.concat([df_train, df_transformed_lda], axis=1)

del transformed_lda_loan_use_train, df_transformed_lda
gc.collect()

transformed_lda_loan_use_test = CLF_LDA_LOAN_USE.transform(bow_loan_use_test)
df_transformed_lda = pd.DataFrame(data=transformed_lda_loan_use_test, columns=LIST_LDA_LOAN_USE)
df_test = pd.concat([df_test, df_transformed_lda], axis=1)

del transformed_lda_loan_use_test, df_transformed_lda
gc.collect()

24

In [10]:
# ラベルエンコードを適用
LIST_LABEL_ENC = [COL_SECTOR_NAME, COL_CURRENCY_POLICY, COL_CURRENCY, COL_REPAYMENT_INTERVAL, COL_DISTRIBUTION_MODEL, COL_COUNTRY_CODE]
df_train, df_test = preprocessing.label_encoding(df_train, df_test, LIST_LABEL_ENC)

In [11]:
# 不要な列をドロップ
df_train.drop(columns=LIST_COL_DROPS, inplace=True)
df_test.drop(columns=LIST_COL_DROPS, inplace=True)

In [24]:
# 学習に使用するデータを設定
X = df_train.drop(columns=[COL_LOAN_AMOUNT])
y = df_train[COL_LOAN_AMOUNT]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train) 

params = {'objective': 'regression',
 'metric': 'l1',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'feature_pre_filter': False,
 'lambda_l1': 5.484934583591358e-05,
 'lambda_l2': 5.79845953329448e-05,
 'num_leaves': 186,
 'feature_fraction': 1.0,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 20,
 'num_iterations': 200,
 'early_stopping_round': 50}

# LightGBM parameters
# params = {
#         'task': 'train',
#         'boosting_type': 'gbdt',
#         'num_iterations': 100,
#         'objective': 'regression', # 目的 : 回帰  
#         'metric': 'mae', # 評価指標 : mae
# }

# # モデルの学習
model = lgb.train(params,
                  train_set=lgb_train, # トレーニングデータの指定
                  valid_sets=lgb_eval, # 検証データの指定
                  )

[1]	valid_0's l1: 523.538
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l1: 491.986
[3]	valid_0's l1: 464.233
[4]	valid_0's l1: 440.418
[5]	valid_0's l1: 419.748
[6]	valid_0's l1: 402.883
[7]	valid_0's l1: 388.522
[8]	valid_0's l1: 375.822
[9]	valid_0's l1: 365.04
[10]	valid_0's l1: 355.75
[11]	valid_0's l1: 348.248
[12]	valid_0's l1: 341.46
[13]	valid_0's l1: 335.845
[14]	valid_0's l1: 330.769
[15]	valid_0's l1: 326.464
[16]	valid_0's l1: 322.62
[17]	valid_0's l1: 319.616
[18]	valid_0's l1: 317.258
[19]	valid_0's l1: 315.231
[20]	valid_0's l1: 313.594
[21]	valid_0's l1: 311.836
[22]	valid_0's l1: 310.588
[23]	valid_0's l1: 309.488
[24]	valid_0's l1: 308.283
[25]	valid_0's l1: 307.349
[26]	valid_0's l1: 306.482
[27]	valid_0's l1: 305.955
[28]	valid_0's l1: 305.396
[29]	valid_0's l1: 305.036
[30]	valid_0's l1: 304.689
[31]	valid_0's l1: 304.428
[32]	valid_0's l1: 304.149
[33]	valid_0's l1: 303.856
[34]	valid_0's l1: 303.503
[35]	valid_0's l1: 303.124
[36]	va

In [26]:
sample_sub = pd.read_csv('data/sample_submission.csv')
preds = model.predict(df_test)
sample_sub[COL_LOAN_AMOUNT] = preds

sample_sub.to_csv('submit/sample_sub.csv', index=None)

In [27]:
preds

array([ 398.29070837,  358.5514827 ,  404.43291983, ...,  393.83096337,
       2082.79278154,  390.49222726])

In [29]:
sample_sub

,LOAN_ID,LOAN_AMOUNT
0,2041445,398.290708
1,1944435,358.551483
2,2083354,404.432920
3,1993565,1057.215118
4,2064272,371.993223
...,...,...
91817,1993862,292.492362
91818,2015070,180.073461
91819,1950349,393.830963
91820,1921580,2082.792782


In [30]:
sample_sub.to_csv('submit/sample_sub.csv', index=None)

In [16]:
import optuna.integration.lightgbm as lgb

In [18]:
# ベストなパラメータ、途中経過を保存する
params = {
    'objective': 'regression',
    'metric': 'mae',
    "verbosity": -1,
    "boosting_type": "gbdt",
}

best_params, history = {}, []

# LightGBM学習
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=200,
                valid_sets=[lgb_train, lgb_eval],
                early_stopping_rounds=50
               )

best_params = gbm.params
best_params

[I 2021-12-27 10:36:12,334] A new study created in memory with name: no-name-00075ec3-af71-4e0b-a80a-a01bc6672564
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[1]	valid_0's l1: 532.331	valid_1's l1: 528.825
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l1: 505.028	valid_1's l1: 502.01
[3]	valid_0's l1: 480.818	valid_1's l1: 478.523
[4]	valid_0's l1: 460.137	valid_1's l1: 458.681
[5]	valid_0's l1: 442.532	valid_1's l1: 441.654
[6]	valid_0's l1: 427.048	valid_1's l1: 427.037
[7]	valid_0's l1: 414.213	valid_1's l1: 414.876
[8]	valid_0's l1: 402.757	valid_1's l1: 403.635
[9]	valid_0's l1: 392.837	valid_1's l1: 394.164
[10]	valid_0's l1: 384.144	valid_1's l1: 386.246
[11]	valid_0's l1: 376.538	valid_1's l1: 379.185
[12]	valid_0's l1: 370.022	valid_1's l1: 372.977
[13]	valid_0's l1: 364.57	valid_1's l1: 368.026
[14]	valid_0's l1: 359.337	valid_1's l1: 363.111
[15]	valid_0's l1: 353.979	valid_1's l1: 357.954
[16]	valid_0's l1: 350.026	valid_1's l1: 354.291
[17]	valid_0's l1: 346.181	valid_1's l1: 350.901
[18]	valid_0's l1: 341.634	valid_1's l1: 346.634
[19]	valid_0's l1: 338.056	valid_1's l1: 343.428
[20]	valid_0's l1: 

feature_fraction, val_score: 302.363314:  14%|#4        | 1/7 [00:00<00:05,  1.05it/s]

[197]	valid_0's l1: 267.011	valid_1's l1: 302.376
[198]	valid_0's l1: 266.884	valid_1's l1: 302.399
[199]	valid_0's l1: 266.706	valid_1's l1: 302.338
[200]	valid_0's l1: 266.601	valid_1's l1: 302.363
Did not meet early stopping. Best iteration is:
[200]	valid_0's l1: 266.601	valid_1's l1: 302.363
[1]	valid_0's l1: 532.816	valid_1's l1: 529.384
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l1: 505.032	valid_1's l1: 502.115
[3]	valid_0's l1: 480.269	valid_1's l1: 477.907
[4]	valid_0's l1: 459.05	valid_1's l1: 457.232
[5]	valid_0's l1: 441.471	valid_1's l1: 440.397
[6]	valid_0's l1: 425.965	valid_1's l1: 425.815
[7]	valid_0's l1: 412.489	valid_1's l1: 413.052
[8]	valid_0's l1: 401.499	valid_1's l1: 402.631
[9]	valid_0's l1: 392.16	valid_1's l1: 394.002
[10]	valid_0's l1: 384.391	valid_1's l1: 386.821
[11]	valid_0's l1: 376.7	valid_1's l1: 379.527
[12]	valid_0's l1: 370.281	valid_1's l1: 373.497
[13]	valid_0's l1: 364.378	valid_1's l1: 368.006
[14]	valid_0's l1

feature_fraction, val_score: 302.363314:  29%|##8       | 2/7 [00:01<00:04,  1.10it/s]

...

min_data_in_leaf, val_score: 297.748551:  80%|########  | 4/5 [00:06<00:01,  1.62s/it]

[93]	valid_0's l1: 248.724	valid_1's l1: 298.978
[94]	valid_0's l1: 248.411	valid_1's l1: 299.081
[95]	valid_0's l1: 248.043	valid_1's l1: 299.236
[96]	valid_0's l1: 247.684	valid_1's l1: 299.224
[97]	valid_0's l1: 247.295	valid_1's l1: 299.222
[98]	valid_0's l1: 246.943	valid_1's l1: 299.213
[99]	valid_0's l1: 246.534	valid_1's l1: 299.296
[100]	valid_0's l1: 246.171	valid_1's l1: 299.354
Early stopping, best iteration is:
[50]	valid_0's l1: 268.245	valid_1's l1: 298.026
[1]	valid_0's l1: 525.774	valid_1's l1: 523.253
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l1: 492.559	valid_1's l1: 491.532
[3]	valid_0's l1: 463.312	valid_1's l1: 463.85
[4]	valid_0's l1: 437.527	valid_1's l1: 439.792
[5]	valid_0's l1: 415.351	valid_1's l1: 419.557
[6]	valid_0's l1: 396.332	valid_1's l1: 402.4
[7]	valid_0's l1: 379.889	valid_1's l1: 387.8
[8]	valid_0's l1: 365.705	valid_1's l1: 375.324
[9]	valid_0's l1: 353.386	valid_1's l1: 364.457
[10]	valid_0's l1: 342.707	valid_1'

min_data_in_leaf, val_score: 297.748551: 100%|##########| 5/5 [00:08<00:00,  1.80s/it]

[183]	valid_0's l1: 191.781	valid_1's l1: 298.506
[184]	valid_0's l1: 191.427	valid_1's l1: 298.495
[185]	valid_0's l1: 191.063	valid_1's l1: 298.564
[186]	valid_0's l1: 190.789	valid_1's l1: 298.534
Early stopping, best iteration is:
[136]	valid_0's l1: 207.572	valid_1's l1: 297.871


{'objective': 'regression',
 'metric': 'l1',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'feature_pre_filter': False,
 'lambda_l1': 5.484934583591358e-05,
 'lambda_l2': 5.79845953329448e-05,
 'num_leaves': 186,
 'feature_fraction': 1.0,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 20,
 'num_iterations': 200,
 'early_stopping_round': 50}

In [19]:
best_params

{'objective': 'regression',
 'metric': 'l1',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'feature_pre_filter': False,
 'lambda_l1': 5.484934583591358e-05,
 'lambda_l2': 5.79845953329448e-05,
 'num_leaves': 186,
 'feature_fraction': 1.0,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 20,
 'num_iterations': 200,
 'early_stopping_round': 50}